# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [12]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [13]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [14]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [15]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [16]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [17]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [18]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [19]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [20]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is continuously monitoring and analyzing market data for AAPL in order to identify trends and predict market movements. It's crucial to use statistical modeling and machine learning to get accurate insights.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for AAPL. Make sure to analyze the data thoroughly and provide insights and alerts about significant market opportunities or threats."}

> Entering new CrewAgentExecutor chain...
I need to start by gathering real-time ma

Final Answer: 
After analyzing the real-time market data for AAPL from Yahoo Finance, it is observed that Apple Inc. (AAPL) is currently trading at $248.17 with a slight increase of 0.05%. The stock price reflects the trading activity during the overnight session on the Blue Ocean ATS. AAPL has a market cap of $3.665 trillion and operates in the consumer electronics/technology sector. The company offers a range of products including iPhones, Mac computers, iPads, wearables, and accessories. AAPL's profitability metrics indicate a healthy profit margin of 26.92% and a return on equity of 171.42%. Analyst recommendations for AAPL vary from Strong Buy to Hold, with a current price target of $248.04. Overall, AAPL has shown positive returns over various time frames and continues to be a key player in the consumer electronics industry.

> Finished chain.
 

After analyzing the real-time market data for AAPL from Yahoo Finance, it is observed that Apple Inc. (AAPL) is currently trading at $2

Final Answer: 
Based on the information gathered from Yahoo Finance, AAPL's recent performance shows a slight increase of 0.05% with the stock price at $248.17. The company has a market cap of $3.665 trillion and operates in the consumer electronics/technology sector. AAPL offers a range of products including iPhones, Mac computers, iPads, wearables, and accessories. The company has a healthy profit margin of 26.92% and a return on equity of 171.42%. Analyst recommendations for AAPL vary from Strong Buy to Hold, with a current price target of $248.04. Overall, AAPL has shown positive returns over various time frames and continues to be a key player in the consumer electronics industry.

To develop potential trading strategies for AAPL based on user's risk tolerance and trading preferences for day trading, considering AAPL's strong market position and positive performance indicators, a strategy that leverages short-term price movements and market volatility could be beneficial. Implemen

Final Answer: Here is the approved trading strategy for AAPL that I found through my search:

Title: Apple Stock: 3 Simple Day Trading Strategies - TradingSim
Link: https://app.tradingsim.com/blog/apple-stock-3-simple-day-trading-strategies/
Snippet: The two-day high-low strategy is ideal for short-term speculative trading or day traders. In this trading strategy, the basic premise is to buy the breakout.

This strategy seems promising for day trading AAPL stock based on its simplicity and focus on short-term trading. I will further analyze this strategy to determine the best execution methods.

> Finished chain.
 

Here is the approved trading strategy for AAPL that I found through my search:

Title: Apple Stock: 3 Simple Day Trading Strategies - TradingSim
Link: https://app.tradingsim.com/blog/apple-stock-3-simple-day-trading-strategies/
Snippet: The two-day high-low strategy is ideal for short-term speculative trading or day traders. In this trading strategy, the basic premise is to

Thought: I now can give a great answer
Final Answer: After searching the internet for information on the Two-Day High-Low Strategy for AAPL, I found various resources discussing similar trading strategies and their risk assessment. While I couldn't access the specific detailed execution plan for AAPL, the general premise of buying breakouts from high or low points in a short-term trading strategy introduces risks associated with market volatility and price movements. Evaluating these risks requires a thorough analysis of historical price data, market conditions, and potential external factors that could impact AAPL's stock performance. It is essential to consider the potential downsides and implement risk management strategies such as stop-loss orders and profit targets to mitigate risks effectively.

> Finished chain.
 

After searching the internet for information on the Two-Day High-Low Strategy for AAPL, I found various resources discussing similar trading strategies and their risk

- Display the final result as Markdown.

In [21]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Two-Day High-Low Strategy for Day Trading AAPL Stock:

After evaluating the proposed Two-Day High-Low Strategy for day trading AAPL stock, the following risks and mitigation strategies have been identified:

1. Market Volatility Risk:
   - Risk: The strategy relies on identifying potential breakouts based on high-low price movements, which can be highly volatile, leading to unexpected price fluctuations.
   - Mitigation: Implement strict stop-loss orders to limit potential losses in case of sudden price reversals. Monitor market conditions closely and adjust entry and exit points accordingly.

2. Execution Risk:
   - Risk: Execution of the strategy may be challenging due to fast-paced price movements and market liquidity issues.
   - Mitigation: Utilize limit orders to ensure timely execution at desired price levels. Regularly review and adjust the execution plan based on real-time market data to minimize execution risks.

3. External Factors Risk:
   - Risk: External factors such as economic news, company announcements, or macroeconomic events can impact AAPL stock prices and disrupt the execution plan.
   - Mitigation: Stay informed about potential market-moving events and adjust trading strategy accordingly. Consider setting alerts for key news updates to react promptly to unexpected developments.

4. Profitability Risk:
   - Risk: Failure to set clear profit targets based on intraday price fluctuations may result in missed opportunities or excessive risk-taking.
   - Mitigation: Establish realistic profit targets based on historical price data and market analysis. Regularly review and adjust profit targets to optimize profitability while managing risk effectively.

In conclusion, the Two-Day High-Low Strategy for day trading AAPL stock presents several risks that need to be carefully evaluated and managed to ensure successful trading outcomes. By implementing the suggested mitigation strategies and closely monitoring market conditions, day traders can optimize profitability while minimizing potential risks associated with the trading strategy.